<a href="https://colab.research.google.com/github/FrkAk/Deep_Learning_Workshop/blob/master/Workshop/Default_Payment_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

## Upload and Preview the Dataset

In [49]:
data = pd.read_csv(
    "dataset.csv",skiprows=1,
    names=["LIMIT_BAL",	"SEX",	"EDUCATION",	"MARRIAGE",	"AGE",	
           "PAY_0",	"PAY_2",	"PAY_3",	"PAY_4"	,"PAY_5",	"PAY_6",
           "BILL_AMT1",	"BILL_AMT2",	"BILL_AMT3",	"BILL_AMT4",	"BILL_AMT5",	"BILL_AMT6",
           "PAY_AMT1",	"PAY_AMT2",	"PAY_AMT3",	"PAY_AMT4",	"PAY_AMT5",	"PAY_AMT6",
           "default.payment.next.month"])

data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
3,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
4,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Split the Dataset

In [50]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)
train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
28397,170000.0,2,2,1,31,-1,0,0,0,0,...,63000.0,0.0,0.0,5000.0,5000.0,4000.0,0.0,0.0,3300.0,0
24673,20000.0,1,2,2,43,0,0,2,0,0,...,16862.0,17197.0,17558.0,1815.0,1000.0,1279.0,614.0,637.0,2350.0,1
26246,130000.0,2,2,2,30,0,0,0,0,0,...,84132.0,79043.0,78205.0,4224.0,4415.0,3600.0,5800.0,2000.0,2005.0,0
12211,80000.0,2,2,1,39,0,0,0,0,0,...,29629.0,28241.0,28030.0,1560.0,1421.0,1001.0,1000.0,1008.0,964.0,0
8065,20000.0,2,2,1,51,-1,-1,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,0


### Extract Labels

In [51]:
train_features = train.copy()
train_labels = train.pop('default.payment.next.month')

test_features = test.copy()
test_labels = test.pop('default.payment.next.month')

### Convert to NumPy array

In [ ]:
train_features = np.array(train_features)
test_features = np.array(test_features)

## Model Structure

In [66]:
model = tf.keras.Sequential([
  layers.Conv1D(filters=64, kernel_size=3,
                strides=1,
                activation="relu",
                padding='causal',
                input_shape=[24, 1]),
  layers.Dense(128,activation='relu'),
  layers.Dense(64,activation='relu'),
  layers.Dense(32,activation='relu'),
  layers.Dense(16,activation='relu'),
  layers.Dense(1,activation='sigmoid')
])
model.summary()



Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 24, 64)            256       
                                                                 
 dense_44 (Dense)            (None, 24, 128)           8320      
                                                                 
 dense_45 (Dense)            (None, 24, 64)            8256      
                                                                 
 dense_46 (Dense)            (None, 24, 32)            2080      
                                                                 
 dense_47 (Dense)            (None, 24, 16)            528       
                                                                 
 dense_48 (Dense)            (None, 24, 1)             17        
                                                                 
Total params: 19,457
Trainable params: 19,457
Non-tra

## Tune the Learning Rate

It is needed to pick an optimal learning rate.

In [ ]:
# Get initial weights
init_weights = model.get_weights()

In [68]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam()

# Set the training parameters
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                  optimizer = optimizer,metrics=['accuracy'])

# Train the model
history = model.fit(train_features,train_labels, epochs=20, callbacks=[lr_schedule])

Epoch 1/100
750/750 [==============================] - 15s 12ms/step - loss: 2.2043 - accuracy: 0.7765 - lr: 1.0000e-08
Epoch 2/100
750/750 [==============================] - 7s 10ms/step - loss: 2.2037 - accuracy: 0.7765 - lr: 1.1220e-08
Epoch 3/100
750/750 [==============================] - 7s 10ms/step - loss: 2.2031 - accuracy: 0.7764 - lr: 1.2589e-08
Epoch 4/100
750/750 [==============================] - 5s 7ms/step - loss: 2.2025 - accuracy: 0.7764 - lr: 1.4125e-08
Epoch 5/100
750/750 [==============================] - 5s 7ms/step - loss: 2.2018 - accuracy: 0.7764 - lr: 1.5849e-08
Epoch 6/100
750/750 [==============================] - 5s 7ms/step - loss: 2.2010 - accuracy: 0.7764 - lr: 1.7783e-08
Epoch 7/100
750/750 [==============================] - 5s 7ms/step - loss: 2.2001 - accuracy: 0.7763 - lr: 1.9953e-08
Epoch 8/100
750/750 [==============================] - 5s 7ms/step - loss: 2.1993 - accuracy: 0.7763 - lr: 2.2387e-08
Epoch 9/100
750/750 [==============================]

In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')

# Set the plot boundaries
plt.axis([1e-8, 1e-3, 0, 100])

## Train the model

In [ ]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

# Set the learning rate
learning_rate = 8e-7

In [ ]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])

In [ ]:
history = model.fit(train_features, train_labels, epochs=10)

In [46]:
model.evaluate(test_features, test_labels)

188/188 [==============================] - 4s 20ms/step - loss: 1.0459


1.0458765029907227